# Hash length extension attack example

A [length extension
attack](https://en.wikipedia.org/wiki/Length_extension_attack) allows an
attacker that intercepted a hash of a message `H(message)` to produce `H(message
|| extension)` without knowing the original message. If the hash is used as a
basic Message Authentication Code by prepending a secret key `H(key ||
message)`, this attack can be used to generate a valid MAC for a new message
`message || pad || extension`, where `pad` is a especially crafted string and
`extension` is completely attacker-controlled.

An example of this attack on the MD5 hash function can be found
in [this article](https://justcryptography.com/length-extension-attack/).
A slightly modified version can be found below to play with.

First, we need an implementation of MD5. The article and this example use the
`pymd5` module below, as it exposes some internal methods that are useful to
carry out this attack.

(If you're reading this in the documentation, the module code has been hidden as
it's quite long. The full version of this notebook can be found in the source
code of the documentation.)

In [1]:
#!/usr/bin/env python3
#
# Derived from:
#
# MD5C.C - RSA Data Security, Inc., MD5 message-digest algorithm
#
# Copyright (C) 1991-2, RSA Data Security, Inc. Created 1991. All
# rights reserved.
#
# License to copy and use this software is granted provided that it
# is identified as the "RSA Data Security, Inc. MD5 Message-Digest
# Algorithm" in all material mentioning or referencing this software
# or this function.
#
# License is also granted to make and use derivative works provided
# that such works are identified as "derived from the RSA Data
# Security, Inc. MD5 Message-Digest Algorithm" in all material
# mentioning or referencing the derived work.
#
# RSA Data Security, Inc. makes no representations concerning either
# the merchantability of this software or the suitability of this
# software for any particular purpose. It is provided "as is"
# without express or implied warranty of any kind.
#
# These notices must be retained in any copies of any part of this
# documentation and/or software.

__doc__ = """pymd5 module - The MD5 hash function in pure Python.

md5(string='', state=None, count=0) - Returns a new md5 objects and
        processes string.  Optional advanced parameters allow you to
        resume an earlier computation by setting the internal state of
        the function and the counter of message bits processed so far.

Most of the interface matches Python's standard hashlib.

md5 objects have these methods and attributes:

 - update(arg): Update the md5 object with the string arg. Repeated calls
                are equivalent to a single call with the concatenation of all
                the arguments.
 - digest():    Return the digest of the strings passed to the update() method
                so far. This may contain non-ASCII characters, including
                NUL bytes.
 - hexdigest(): Like digest() except the digest is returned as a string of
                double length, containing only hexadecimal digits.

 - digest_size: The size of the resulting hash in bytes (16).
 - block_size:  The internal block size of the hash algorithm in bytes (64).

For example, to obtain the digest of the string 'Nobody inspects the
spammish repetition':

    >>> import pymd5
    >>> m = pymd5.md5()
    >>> m.update("Nobody inspects")
    >>> m.update(" the spammish repetition")
    >>> m.digest()

More condensed:

    >>> pymd5.md5("Nobody inspects the spammish repetition").hexdigest()
    'bb649c83dd1ea5c9d9dec9a18df0ffe9'


The module also exposes two low-level methods to help with crypto
experiments:

 - md5_compress(state, block): The MD5 compression function; returns a
                               new 16-byte state based on the 16-byte
                               previous state and a 512-byte message
                               block.

 - padding(msg_bits):          Generate the padding that should be appended
                               to the end of a message of the given size to
                               reach a multiple of the block size.


"""

# Constants for compression function.

S11 = 7
S12 = 12
S13 = 17
S14 = 22
S21 = 5
S22 = 9
S23 = 14
S24 = 20
S31 = 4
S32 = 11
S33 = 16
S34 = 23
S41 = 6
S42 = 10
S43 = 15
S44 = 21

PADDING = b"\x80" + 63 * b"\0"


# F, G, H and I: basic MD5 functions.
def F(x, y, z):
    return ((x) & (y)) | ((~x) & (z))


def G(x, y, z):
    return ((x) & (z)) | ((y) & (~z))


def H(x, y, z):
    return (x) ^ (y) ^ (z)


def I(x, y, z):
    return (y) ^ ((x) | (~z))


def ROTATE_LEFT(x, n):
    x = x & 0xFFFFFFFF  # make shift unsigned
    return (((x) << (n)) | ((x) >> (32 - (n)))) & 0xFFFFFFFF


# FF, GG, HH, and II transformations for rounds 1, 2, 3, and 4.
# Rotation is separate from addition to prevent recomputation.


def FF(a, b, c, d, x, s, ac):
    a = a + F((b), (c), (d)) + (x) + (ac)
    a = ROTATE_LEFT((a), (s))
    a = a + b
    return a  # must assign this to a


def GG(a, b, c, d, x, s, ac):
    a = a + G((b), (c), (d)) + (x) + (ac)
    a = ROTATE_LEFT((a), (s))
    a = a + b
    return a  # must assign this to a


def HH(a, b, c, d, x, s, ac):
    a = a + H((b), (c), (d)) + (x) + (ac)
    a = ROTATE_LEFT((a), (s))
    a = a + b
    return a  # must assign this to a


def II(a, b, c, d, x, s, ac):
    a = a + I((b), (c), (d)) + (x) + (ac)
    a = ROTATE_LEFT((a), (s))
    a = a + b
    return a  # must assign this to a


class md5(object):
    digest_size = 16  # size of the resulting hash in bytes
    block_size = 64  # hash algorithm's internal block size

    def __init__(self, string="", state=None, count=0):
        """md5(string='', state=None, count=0) - Return a new md5
        hash object, optionally initialized to a given internal state
        and count of message bits processed so far, then processes
        string.
        """
        self.count = 0
        self.buffer = b""

        if state is None:
            # initial state defined by standard
            self.state = (
                0x67452301,
                0xEFCDAB89,
                0x98BADCFE,
                0x10325476,
            )
        else:
            self.state = _decode(state, md5.digest_size)
        if count is not None:
            self.count = count
        if string:
            self.update(string)

    def update(self, input):
        """update(input) - Update the md5 object with the string
        arg. Repeated calls are equivalent to a single call with the
        concatenation of all the arguments.
        """
        if not isinstance(input, bytes):
            input = input.encode("utf-8")
        inputLen = len(input)
        index = int(self.count >> 3) & 0x3F
        self.count = self.count + (inputLen << 3)  # update number of bits
        partLen = md5.block_size - index

        # apply compression function to as many blocks as we have
        if inputLen >= partLen:
            self.buffer = self.buffer[:index] + input[:partLen]
            self.state = md5_compress(self.state, self.buffer)
            i = partLen
            while i + 63 < inputLen:
                self.state = md5_compress(self.state, input[i : i + md5.block_size])
                i = i + md5.block_size
            index = 0
        else:
            i = 0

        # buffer remaining output
        self.buffer = self.buffer[:index] + input[i:inputLen]

    def digest(self):
        """digest() - Return the MD5 hash of the strings passed to the
        update() method so far. This is a string of digest_size bytes
        which may contain non-ASCII characters, including null bytes.
        """
        _buffer, _count, _state = self.buffer, self.count, self.state
        self.update(padding(self.count))
        result = self.state
        self.buffer, self.count, self.state = _buffer, _count, _state
        return _encode(result, md5.digest_size)

    def hexdigest(self):
        """hexdigest() - Like digest() except the hash value is
        returned as a string of hexadecimal digits.
        """
        return self.digest().hex()


## end of class


def padding(msg_bits):
    """padding(msg_bits) - Generates the padding that should be
    appended to the end of a message of the given size to reach
    a multiple of the block size."""

    index = int((msg_bits >> 3) & 0x3F)
    if index < 56:
        padLen = 56 - index
    else:
        padLen = 120 - index

    # (the last 8 bytes store the number of bits in the message)
    return PADDING[:padLen] + _encode((msg_bits & 0xFFFFFFFF, msg_bits >> 32), 8)


def md5_compress(state, block):
    """md5_compress(state, block) - The MD5 compression function.
    Outputs a 16-byte state based on a 16-byte previous state and a
    512-byte message block.
    """
    a, b, c, d = state
    x = _decode(block, md5.block_size)

    #  Round
    a = FF(a, b, c, d, x[0], S11, 0xD76AA478)  # 1
    d = FF(d, a, b, c, x[1], S12, 0xE8C7B756)  # 2
    c = FF(c, d, a, b, x[2], S13, 0x242070DB)  # 3
    b = FF(b, c, d, a, x[3], S14, 0xC1BDCEEE)  # 4
    a = FF(a, b, c, d, x[4], S11, 0xF57C0FAF)  # 5
    d = FF(d, a, b, c, x[5], S12, 0x4787C62A)  # 6
    c = FF(c, d, a, b, x[6], S13, 0xA8304613)  # 7
    b = FF(b, c, d, a, x[7], S14, 0xFD469501)  # 8
    a = FF(a, b, c, d, x[8], S11, 0x698098D8)  # 9
    d = FF(d, a, b, c, x[9], S12, 0x8B44F7AF)  # 10
    c = FF(c, d, a, b, x[10], S13, 0xFFFF5BB1)  # 11
    b = FF(b, c, d, a, x[11], S14, 0x895CD7BE)  # 12
    a = FF(a, b, c, d, x[12], S11, 0x6B901122)  # 13
    d = FF(d, a, b, c, x[13], S12, 0xFD987193)  # 14
    c = FF(c, d, a, b, x[14], S13, 0xA679438E)  # 15
    b = FF(b, c, d, a, x[15], S14, 0x49B40821)  # 16

    # Round 2
    a = GG(a, b, c, d, x[1], S21, 0xF61E2562)  # 17
    d = GG(d, a, b, c, x[6], S22, 0xC040B340)  # 18
    c = GG(c, d, a, b, x[11], S23, 0x265E5A51)  # 19
    b = GG(b, c, d, a, x[0], S24, 0xE9B6C7AA)  # 20
    a = GG(a, b, c, d, x[5], S21, 0xD62F105D)  # 21
    d = GG(d, a, b, c, x[10], S22, 0x2441453)  # 22
    c = GG(c, d, a, b, x[15], S23, 0xD8A1E681)  # 23
    b = GG(b, c, d, a, x[4], S24, 0xE7D3FBC8)  # 24
    a = GG(a, b, c, d, x[9], S21, 0x21E1CDE6)  # 25
    d = GG(d, a, b, c, x[14], S22, 0xC33707D6)  # 26
    c = GG(c, d, a, b, x[3], S23, 0xF4D50D87)  # 27
    b = GG(b, c, d, a, x[8], S24, 0x455A14ED)  # 28
    a = GG(a, b, c, d, x[13], S21, 0xA9E3E905)  # 29
    d = GG(d, a, b, c, x[2], S22, 0xFCEFA3F8)  # 30
    c = GG(c, d, a, b, x[7], S23, 0x676F02D9)  # 31
    b = GG(b, c, d, a, x[12], S24, 0x8D2A4C8A)  # 32

    # Round 3
    a = HH(a, b, c, d, x[5], S31, 0xFFFA3942)  # 33
    d = HH(d, a, b, c, x[8], S32, 0x8771F681)  # 34
    c = HH(c, d, a, b, x[11], S33, 0x6D9D6122)  # 35
    b = HH(b, c, d, a, x[14], S34, 0xFDE5380C)  # 36
    a = HH(a, b, c, d, x[1], S31, 0xA4BEEA44)  # 37
    d = HH(d, a, b, c, x[4], S32, 0x4BDECFA9)  # 38
    c = HH(c, d, a, b, x[7], S33, 0xF6BB4B60)  # 39
    b = HH(b, c, d, a, x[10], S34, 0xBEBFBC70)  # 40
    a = HH(a, b, c, d, x[13], S31, 0x289B7EC6)  # 41
    d = HH(d, a, b, c, x[0], S32, 0xEAA127FA)  # 42
    c = HH(c, d, a, b, x[3], S33, 0xD4EF3085)  # 43
    b = HH(b, c, d, a, x[6], S34, 0x4881D05)  # 44
    a = HH(a, b, c, d, x[9], S31, 0xD9D4D039)  # 45
    d = HH(d, a, b, c, x[12], S32, 0xE6DB99E5)  # 46
    c = HH(c, d, a, b, x[15], S33, 0x1FA27CF8)  # 47
    b = HH(b, c, d, a, x[2], S34, 0xC4AC5665)  # 48

    # Round 4
    a = II(a, b, c, d, x[0], S41, 0xF4292244)  # 49
    d = II(d, a, b, c, x[7], S42, 0x432AFF97)  # 50
    c = II(c, d, a, b, x[14], S43, 0xAB9423A7)  # 51
    b = II(b, c, d, a, x[5], S44, 0xFC93A039)  # 52
    a = II(a, b, c, d, x[12], S41, 0x655B59C3)  # 53
    d = II(d, a, b, c, x[3], S42, 0x8F0CCC92)  # 54
    c = II(c, d, a, b, x[10], S43, 0xFFEFF47D)  # 55
    b = II(b, c, d, a, x[1], S44, 0x85845DD1)  # 56
    a = II(a, b, c, d, x[8], S41, 0x6FA87E4F)  # 57
    d = II(d, a, b, c, x[15], S42, 0xFE2CE6E0)  # 58
    c = II(c, d, a, b, x[6], S43, 0xA3014314)  # 59
    b = II(b, c, d, a, x[13], S44, 0x4E0811A1)  # 60
    a = II(a, b, c, d, x[4], S41, 0xF7537E82)  # 61
    d = II(d, a, b, c, x[11], S42, 0xBD3AF235)  # 62
    c = II(c, d, a, b, x[2], S43, 0x2AD7D2BB)  # 63
    b = II(b, c, d, a, x[9], S44, 0xEB86D391)  # 64

    return (
        0xFFFFFFFF & (state[0] + a),
        0xFFFFFFFF & (state[1] + b),
        0xFFFFFFFF & (state[2] + c),
        0xFFFFFFFF & (state[3] + d),
    )


import struct, string


def _encode(input, len):
    k = len // 4
    res = struct.pack("<%iI" % k, *(list(input[:k])))
    return res


def _decode(input, len):
    k = len // 4
    res = struct.unpack("<%iI" % k, input[:len])
    return list(res)


def test(input=""):
    """test(input): displays results of input hashed with our md5
    function and the standard Python hashlib implementation
    """
    print(md5(input).hexdigest())
    import hashlib

    print(hashlib.md5(input.encode("utf-8")).hexdigest())


if __name__ == "__main__":
    test("crypt")

f7bd616b6c841d2538735f76d1e02b57
f7bd616b6c841d2538735f76d1e02b57


Then we define a password and a message to illustrate this.

In [2]:
# The secret password used to validate the message.
password = "super secret password"
# A super important message.
message = "Bring two croissants and a brioche"

We also compute our legitimate MAC.

In [3]:
mac = md5(password + message).digest()

Now we define a function to test if a digest matches a given message.

In [4]:
def verify(msg: bytes, digest: bytes) -> bool:
    """Checks that the digest is valid for the given message.

    Args:
        msg: The received message.
        digest: The accompanying digest that should correspond to
            MD5(password || msg).

    Returns:
        True if the digest is valid for the given message, i.e.
        MD5(password || msg) == digest.
    """
    h = md5()
    h.update(password.encode() + msg)
    return h.digest() == digest

It's time to implement the attack.

We first need to know the length of `password || message`. In this case we
assume that we know the actual message sent *and* the length of the password
(but not the password itself). If we didn't know the length, we would have to
test different lengths until one passes `verify`.

```python
input_len = password_len + len(message)
```

From this, we can deduce how many bits have already been processed by the
algorithm. This is the length of the message plus the length of the padding that
was added by the algorithm. The pymd5 implementation exposes the `padding`
function that returns the padding that is added depending on the length of
input.

```python
processed_bits = (input_len + len(padding(input_len * 8))) * 8
```

We can now initialize a new `md5` object using the known state and the number of
processed bits:

```python
h = md5(state=digest, count=processed_bits)
```

We update this new instance with the message of our choice and produce the new
digest, which is our new MAC:

```python
h.update(b"and 42 pains au chocolat")
new_mac = h.digest()
```

Since we know the original message, we compute the new message that corresponds
to our new digest:

```python
new_message = message.encode() + padding(input_len * 8) + b"and 42 pains au chocolat"
```

Putting this all together gives us:

In [5]:
def attack(message: str, password_len: int, mac: bytes) -> tuple[bytes, bytes]:
    """Implements a hash length extension attack on MD5.

    Args:
        message: The known message to extend.
        password_len: The length in bytes of the secret password.
        mac: The intercepted MAC == MD5(password || message).

    Returns:
        A tuple containing the new message and the new digest.
    """
    input_len = password_len + len(message)
    processed_bits = (input_len + len(padding(input_len * 8))) * 8

    h = md5(state=mac, count=processed_bits)

    h.update(b"and 42 pains au chocolat")

    new_mac = h.digest()

    new_message = (
        message.encode() + padding(input_len * 8) + b"and 42 pains au chocolat"
    )

    return new_message, new_mac

We can now test the attack:

In [6]:
password_length = len(password)

new_message, new_mac = attack(message, password_length, mac)
if verify(new_message, new_mac):
    print("Success!")
    print(f"{message = }")
    print(f"{mac.hex() = }")
    print(f"{new_message = }")
    print(f"{new_mac.hex() = }")
else:
    print("Failed!")

Success!
message = 'Bring two croissants and a brioche'
mac.hex() = '356177536596b1dda0e9767180683aea'
new_message = b'Bring two croissants and a brioche\x80\xb8\x01\x00\x00\x00\x00\x00\x00and 42 pains au chocolat'
new_mac.hex() = 'ce6c189011a7a8508a59e7f33104d401'


We see that while the new message has some "garbage" in the middle, it passes
the verification! Hopefully the receiver will think that it's just a display bug
and fulfils our order. ;)

Side-note: while the introduction says that knowledge of the message is not
required, we did use it to compute the new message to verify. This is because of
the example we used to illustrate this attack that emulates a possible use of
this construction. Without knowledge of the message, we can still craft a valid
hash as long as we know/can guess the length of the password and the message,
but then the receiver in our example would have no way of verifying this. :)